# 1. Tracking simple objects in videos

In [12]:
import cv2  
import numpy as np  
import matplotlib.pyplot as plt

In [13]:
video_path = 'video sequences/synthetic/escrime-4-3.avi'  # Chemin vers la vidéo
N = 500  # Nombre de particules
sigma = np.diag([40, 40])  # Bruit gaussien pour la prédiction (modèle de transition)
lambda_likelihood = 1  # Paramètre \lambda pour le calcul de la vraisemblance
hist_bins = 256  # Nombre de bins pour l'histogramme de couleurs

In [14]:
cap = cv2.VideoCapture(video_path)

In [15]:
ret, frame = cap.read()
if not ret:
    print("Erreur : Impossible de lire la vidéo")
else:
    # Sélectionner la zone à suivre manuellement
    roi = cv2.selectROI(frame)
    x, y, w, h = roi
    cv2.destroyWindow("ROI Selector")

In [16]:
def calc_histogram(image, roi):
    x, y, w, h = roi
    roi_image = image[int(y):int(y+h), int(x):int(x+w)]
    hist = cv2.calcHist([roi_image], [0], None, [hist_bins], [0, 256])
    hist = cv2.normalize(hist, hist).flatten()
    return hist

# Calculer l'histogramme de la région initiale
roi_hist = calc_histogram(frame, roi)

print(w,h,x,y)

33 29 305 226


In [17]:
particles = np.column_stack((
    np.random.uniform(x, x + w, N),
    np.random.uniform(y, y + h, N)
))
weights = np.ones(N) / N  # Poids initiaux uniformes
print(particles)

[[335.20363741 234.21957761]
 [315.85593651 239.58159728]
 [316.15002293 248.88682368]
 [319.67462202 248.93618137]
 [325.61793    245.77738467]
 [328.45952119 247.61820348]
 [308.17211922 252.58170739]
 [328.81202694 247.49154646]
 [330.2701893  243.85921236]
 [322.19017402 252.41722227]
 [335.73941825 240.49078034]
 [321.64450129 254.91212889]
 [336.62180679 232.03155063]
 [315.12412119 253.22927149]
 [310.38587404 240.22413197]
 [329.27760264 237.69351449]
 [327.71159518 235.43659411]
 [335.59883379 236.90566965]
 [308.55277786 248.22663041]
 [318.61469311 238.55862311]
 [325.79092082 248.62348189]
 [323.3653673  240.33744547]
 [324.27359102 250.09612584]
 [318.84979793 250.7541246 ]
 [331.69268627 254.81979997]
 [316.17668324 245.58240062]
 [307.22265351 227.98060905]
 [320.17497488 244.41653209]
 [325.97965825 237.67999288]
 [315.64331124 237.5029725 ]
 [332.15911758 254.83447714]
 [330.94957238 236.59094055]
 [322.49668756 227.25533476]
 [320.70658828 252.97991764]
 [331.39908217

In [18]:
def predict(particles, sigma):
    noise = np.random.multivariate_normal([0, 0], sigma, N)
    particles += noise
    return particles

In [19]:
def compute_weights(particles, frame, roi_hist, lambd=lambda_likelihood):
    weights = np.zeros(N)
    for i, particle in enumerate(particles):
        px, py = particle
        # Définir une petite zone autour de chaque particule
        patch_hist = calc_histogram(frame, (px, py, w, h))

        # Calculer la distance entre les deux histogrammes 
        dist = cv2.compareHist(roi_hist, patch_hist, cv2.HISTCMP_BHATTACHARYYA)

        # Mettre à jour les poids selon la fonction de vraisemblance
        weights[i] = np.exp(-lambd * dist**2)

    # Normaliser les poids pour qu'ils forment une distribution de probabilité
    weights /= np.sum(weights)
    return weights

In [20]:
def resample(particles, weights):
    """
    Rééchantillonne les particules en fonction de leurs poids.
    
    Parameters:
    particles (numpy.ndarray): Les particules (positions) actuelles, de taille (N, 2).
    weights (numpy.ndarray): Les poids actuels des particules, de taille (N,).
    
    Returns:
    numpy.ndarray: Les nouvelles particules rééchantillonnées, de taille (N, 2).
    """
    N = len(particles)
    indices = np.arange(N)

    # Choisir des particules selon leurs poids en utilisant la méthode du rééchantillonnage systématique
    cumulative_sum = np.cumsum(weights)
    cumulative_sum[-1] = 1.0  # Pour s'assurer que la somme est exactement 1
    u0 = np.random.uniform(0, 1/N)
    positions = (u0 + np.arange(N) / N)  # Échantillons également espacés

    # Indices des particules sélectionnées
    new_indices = np.searchsorted(cumulative_sum, positions)

    # Créer un nouvel ensemble de particules rééchantillonnées
    resampled_particles = particles[new_indices]

    return resampled_particles

What if we skip on frame for every two consecutive frames ?

In [21]:
frame_skip = 1  # Sauter une frame sur deux

In [22]:
# Boucle principale avec frame_skip pour ignorer des frames
while True:
    for _ in range(frame_skip):  # Sauter des frames si nécessaire
        ret, frame = cap.read()
        if not ret:
            break 

    # Prédiction : mise à jour des particules avec du bruit
    particles = predict(particles, sigma)

    # Correction : calcul des poids à partir des histogrammes
    weights = compute_weights(particles, frame, roi_hist)

    # Rééchantillonnage : sélection des particules en fonction des poids
    particles = resample(particles, weights)

    # Calculer la position moyenne de l'objet après rééchantillonnage
    estimated_position = np.average(particles, axis=0, weights=weights)

    # Dessiner le rectangle autour de la position estimée
    cv2.rectangle(frame,
                  (int(estimated_position[0]), int(estimated_position[1])),
                  (int(estimated_position[0] + w), int(estimated_position[1] + h)),
                  (0, 255, 0), 2)

    # Afficher chaque particule sur la vidéo avec un cercle bleu
    for particle in particles:
        cv2.circle(frame, (int(particle[0]), int(particle[1])), 2, (0, 255, 255), -1)  

    # Afficher l'image avec le suivi
    cv2.imshow("Tracking", frame)

    # Quitter en appuyant sur 'q'
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

# Libérer les ressources
cap.release()
cv2.destroyAllWindows()

TypeError: 'NoneType' object is not subscriptable